In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
x = torch.randn(2, 8, 10)
x , x.shape

(tensor([[[-1.1106,  0.9686,  0.5912, -0.2099,  1.6302, -0.8762, -0.6009,
            1.0252, -0.2825,  0.5011],
          [-1.0471, -1.3095,  0.5677, -1.6786,  1.8040, -0.1652,  0.1316,
           -0.0587,  0.0958,  0.7152],
          [-0.1431, -0.9641, -0.3411, -0.4061,  2.3285, -0.7729,  1.6679,
           -2.0809, -0.8932,  0.4662],
          [-2.6478, -1.3366, -1.6653, -0.3820,  1.4536,  0.3458, -0.5184,
            0.3567, -0.2809,  0.0345],
          [-0.8419,  0.4518, -0.2638,  0.1185,  0.0992,  1.1773,  0.6301,
            1.3826, -1.4353,  0.2172],
          [-0.8487, -0.8153, -0.2547,  0.4806,  2.1043, -1.0114, -0.0605,
           -0.4340, -1.1816, -1.3028],
          [-0.2313, -1.6736,  1.1868,  0.1514,  1.2037,  0.6815,  0.5119,
           -0.8219, -1.0702,  0.9774],
          [-0.7753,  0.4698, -2.0141, -0.4705, -0.6935, -0.5498,  0.6422,
            1.5470,  0.3227,  0.7972]],
 
         [[-2.7740,  0.9726, -0.0049, -1.2102,  2.8382, -0.5756,  1.6773,
            0.1653,

In [4]:
# 门控机制
gate = nn.Linear(10,4)
logits = gate(x)
logits,logits.shape

(tensor([[[-0.7282, -0.8363,  0.1601,  0.9720],
          [-0.6245, -0.4649, -0.0513,  0.8264],
          [-0.6369, -0.2655,  0.4256,  0.3038],
          [-0.5963, -1.6130, -0.0584,  0.1944],
          [ 0.1214, -0.0660, -0.1532,  0.3703],
          [-0.6829, -0.7744, -0.3790,  0.0347],
          [-1.2188,  0.2750, -0.1770,  0.3746],
          [ 0.7845, -1.1465,  0.7213,  0.6494]],
 
         [[-0.4878, -1.1083,  0.7122,  1.1838],
          [-0.6631,  0.3346,  0.9861,  1.6595],
          [-0.9269, -0.6006,  0.1411,  0.6790],
          [ 0.4634, -0.6352, -0.0380, -0.0348],
          [ 0.8594,  0.3049, -0.0748,  0.8208],
          [-0.1530, -0.9408, -0.1673, -0.1462],
          [-0.5209, -0.6614,  0.5353,  0.2677],
          [ 0.3297,  0.1474,  0.4207, -0.1707]]], grad_fn=<ViewBackward0>),
 torch.Size([2, 8, 4]))

In [5]:
# 激活两个专家
logits_topk, indices = logits.topk(2, dim=-1)
logits_topk,logits_topk.shape, indices,indices.shape

(tensor([[[ 0.9720,  0.1601],
          [ 0.8264, -0.0513],
          [ 0.4256,  0.3038],
          [ 0.1944, -0.0584],
          [ 0.3703,  0.1214],
          [ 0.0347, -0.3790],
          [ 0.3746,  0.2750],
          [ 0.7845,  0.7213]],
 
         [[ 1.1838,  0.7122],
          [ 1.6595,  0.9861],
          [ 0.6790,  0.1411],
          [ 0.4634, -0.0348],
          [ 0.8594,  0.8208],
          [-0.1462, -0.1530],
          [ 0.5353,  0.2677],
          [ 0.4207,  0.3297]]], grad_fn=<TopkBackward0>),
 torch.Size([2, 8, 2]),
 tensor([[[3, 2],
          [3, 2],
          [2, 3],
          [3, 2],
          [3, 0],
          [3, 2],
          [3, 1],
          [0, 2]],
 
         [[3, 2],
          [3, 2],
          [3, 2],
          [0, 3],
          [0, 3],
          [3, 0],
          [2, 3],
          [2, 0]]]),
 torch.Size([2, 8, 2]))

In [6]:
zeros = torch.full_like(logits, float("-inf"))
zeros,zeros.shape

(tensor([[[-inf, -inf, -inf, -inf],
          [-inf, -inf, -inf, -inf],
          [-inf, -inf, -inf, -inf],
          [-inf, -inf, -inf, -inf],
          [-inf, -inf, -inf, -inf],
          [-inf, -inf, -inf, -inf],
          [-inf, -inf, -inf, -inf],
          [-inf, -inf, -inf, -inf]],
 
         [[-inf, -inf, -inf, -inf],
          [-inf, -inf, -inf, -inf],
          [-inf, -inf, -inf, -inf],
          [-inf, -inf, -inf, -inf],
          [-inf, -inf, -inf, -inf],
          [-inf, -inf, -inf, -inf],
          [-inf, -inf, -inf, -inf],
          [-inf, -inf, -inf, -inf]]]),
 torch.Size([2, 8, 4]))

In [7]:
sparse_logits = zeros.scatter(dim=-1, index=indices, src=logits_topk)
sparse_logits,sparse_logits.shape

(tensor([[[   -inf,    -inf,  0.1601,  0.9720],
          [   -inf,    -inf, -0.0513,  0.8264],
          [   -inf,    -inf,  0.4256,  0.3038],
          [   -inf,    -inf, -0.0584,  0.1944],
          [ 0.1214,    -inf,    -inf,  0.3703],
          [   -inf,    -inf, -0.3790,  0.0347],
          [   -inf,  0.2750,    -inf,  0.3746],
          [ 0.7845,    -inf,  0.7213,    -inf]],
 
         [[   -inf,    -inf,  0.7122,  1.1838],
          [   -inf,    -inf,  0.9861,  1.6595],
          [   -inf,    -inf,  0.1411,  0.6790],
          [ 0.4634,    -inf,    -inf, -0.0348],
          [ 0.8594,    -inf,    -inf,  0.8208],
          [-0.1530,    -inf,    -inf, -0.1462],
          [   -inf,    -inf,  0.5353,  0.2677],
          [ 0.3297,    -inf,  0.4207,    -inf]]], grad_fn=<ScatterBackward0>),
 torch.Size([2, 8, 4]))

In [8]:
sparse_logits = F.softmax(sparse_logits, dim=-1)
sparse_logits,sparse_logits.shape

(tensor([[[0.0000, 0.0000, 0.3075, 0.6925],
          [0.0000, 0.0000, 0.2936, 0.7064],
          [0.0000, 0.0000, 0.5304, 0.4696],
          [0.0000, 0.0000, 0.4371, 0.5629],
          [0.4381, 0.0000, 0.0000, 0.5619],
          [0.0000, 0.0000, 0.3980, 0.6020],
          [0.0000, 0.4751, 0.0000, 0.5249],
          [0.5158, 0.0000, 0.4842, 0.0000]],
 
         [[0.0000, 0.0000, 0.3842, 0.6158],
          [0.0000, 0.0000, 0.3377, 0.6623],
          [0.0000, 0.0000, 0.3687, 0.6313],
          [0.6220, 0.0000, 0.0000, 0.3780],
          [0.5097, 0.0000, 0.0000, 0.4903],
          [0.4983, 0.0000, 0.0000, 0.5017],
          [0.0000, 0.0000, 0.5665, 0.4335],
          [0.4773, 0.0000, 0.5227, 0.0000]]], grad_fn=<SoftmaxBackward0>),
 torch.Size([2, 8, 4]))

In [9]:
gate_logit = logits.view(-1, 4)
gate_logit,gate_logit.shape

(tensor([[-0.7282, -0.8363,  0.1601,  0.9720],
         [-0.6245, -0.4649, -0.0513,  0.8264],
         [-0.6369, -0.2655,  0.4256,  0.3038],
         [-0.5963, -1.6130, -0.0584,  0.1944],
         [ 0.1214, -0.0660, -0.1532,  0.3703],
         [-0.6829, -0.7744, -0.3790,  0.0347],
         [-1.2188,  0.2750, -0.1770,  0.3746],
         [ 0.7845, -1.1465,  0.7213,  0.6494],
         [-0.4878, -1.1083,  0.7122,  1.1838],
         [-0.6631,  0.3346,  0.9861,  1.6595],
         [-0.9269, -0.6006,  0.1411,  0.6790],
         [ 0.4634, -0.6352, -0.0380, -0.0348],
         [ 0.8594,  0.3049, -0.0748,  0.8208],
         [-0.1530, -0.9408, -0.1673, -0.1462],
         [-0.5209, -0.6614,  0.5353,  0.2677],
         [ 0.3297,  0.1474,  0.4207, -0.1707]], grad_fn=<ViewBackward0>),
 torch.Size([16, 4]))

In [10]:
final_outputs = torch.zeros_like(x)
final_outputs,final_outputs.shape

(tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],
 
         [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]]),
 torch.Size([2, 8, 10]))

In [11]:
# 展平
x_flat = x.view(-1, x.shape[-1])
x_flat,x_flat.shape,x.shape

(tensor([[-1.1106,  0.9686,  0.5912, -0.2099,  1.6302, -0.8762, -0.6009,  1.0252,
          -0.2825,  0.5011],
         [-1.0471, -1.3095,  0.5677, -1.6786,  1.8040, -0.1652,  0.1316, -0.0587,
           0.0958,  0.7152],
         [-0.1431, -0.9641, -0.3411, -0.4061,  2.3285, -0.7729,  1.6679, -2.0809,
          -0.8932,  0.4662],
         [-2.6478, -1.3366, -1.6653, -0.3820,  1.4536,  0.3458, -0.5184,  0.3567,
          -0.2809,  0.0345],
         [-0.8419,  0.4518, -0.2638,  0.1185,  0.0992,  1.1773,  0.6301,  1.3826,
          -1.4353,  0.2172],
         [-0.8487, -0.8153, -0.2547,  0.4806,  2.1043, -1.0114, -0.0605, -0.4340,
          -1.1816, -1.3028],
         [-0.2313, -1.6736,  1.1868,  0.1514,  1.2037,  0.6815,  0.5119, -0.8219,
          -1.0702,  0.9774],
         [-0.7753,  0.4698, -2.0141, -0.4705, -0.6935, -0.5498,  0.6422,  1.5470,
           0.3227,  0.7972],
         [-2.7740,  0.9726, -0.0049, -1.2102,  2.8382, -0.5756,  1.6773,  0.1653,
           0.0534,  0.0058],
 

In [12]:
sparse_logits_flat = sparse_logits.view(-1, sparse_logits.shape[-1])
sparse_logits_flat,sparse_logits_flat.shape,sparse_logits.shape

(tensor([[0.0000, 0.0000, 0.3075, 0.6925],
         [0.0000, 0.0000, 0.2936, 0.7064],
         [0.0000, 0.0000, 0.5304, 0.4696],
         [0.0000, 0.0000, 0.4371, 0.5629],
         [0.4381, 0.0000, 0.0000, 0.5619],
         [0.0000, 0.0000, 0.3980, 0.6020],
         [0.0000, 0.4751, 0.0000, 0.5249],
         [0.5158, 0.0000, 0.4842, 0.0000],
         [0.0000, 0.0000, 0.3842, 0.6158],
         [0.0000, 0.0000, 0.3377, 0.6623],
         [0.0000, 0.0000, 0.3687, 0.6313],
         [0.6220, 0.0000, 0.0000, 0.3780],
         [0.5097, 0.0000, 0.0000, 0.4903],
         [0.4983, 0.0000, 0.0000, 0.5017],
         [0.0000, 0.0000, 0.5665, 0.4335],
         [0.4773, 0.0000, 0.5227, 0.0000]], grad_fn=<ViewBackward0>),
 torch.Size([16, 4]),
 torch.Size([2, 8, 4]))

In [13]:
expert_mask = (indices == 0)
indices,indices.shape,expert_mask,expert_mask.shape

(tensor([[[3, 2],
          [3, 2],
          [2, 3],
          [3, 2],
          [3, 0],
          [3, 2],
          [3, 1],
          [0, 2]],
 
         [[3, 2],
          [3, 2],
          [3, 2],
          [0, 3],
          [0, 3],
          [3, 0],
          [2, 3],
          [2, 0]]]),
 torch.Size([2, 8, 2]),
 tensor([[[False, False],
          [False, False],
          [False, False],
          [False, False],
          [False,  True],
          [False, False],
          [False, False],
          [ True, False]],
 
         [[False, False],
          [False, False],
          [False, False],
          [ True, False],
          [ True, False],
          [False,  True],
          [False, False],
          [False,  True]]]),
 torch.Size([2, 8, 2]))

In [14]:
# 每个token 是否用到当前i的专家
expert_mask = expert_mask.any(-1)
expert_mask ,expert_mask.shape

(tensor([[False, False, False, False,  True, False, False,  True],
         [False, False, False,  True,  True,  True, False,  True]]),
 torch.Size([2, 8]))

In [15]:
#展平
expert_mask_flat = expert_mask.view(-1)
expert_mask_flat,expert_mask_flat.shape


(tensor([False, False, False, False,  True, False, False,  True, False, False,
         False,  True,  True,  True, False,  True]),
 torch.Size([16]))

In [16]:
# 取出当前专家需要计算的token
expert_input = x_flat[expert_mask_flat]
x_flat,x_flat.shape,expert_input,expert_input.shape

(tensor([[-1.1106,  0.9686,  0.5912, -0.2099,  1.6302, -0.8762, -0.6009,  1.0252,
          -0.2825,  0.5011],
         [-1.0471, -1.3095,  0.5677, -1.6786,  1.8040, -0.1652,  0.1316, -0.0587,
           0.0958,  0.7152],
         [-0.1431, -0.9641, -0.3411, -0.4061,  2.3285, -0.7729,  1.6679, -2.0809,
          -0.8932,  0.4662],
         [-2.6478, -1.3366, -1.6653, -0.3820,  1.4536,  0.3458, -0.5184,  0.3567,
          -0.2809,  0.0345],
         [-0.8419,  0.4518, -0.2638,  0.1185,  0.0992,  1.1773,  0.6301,  1.3826,
          -1.4353,  0.2172],
         [-0.8487, -0.8153, -0.2547,  0.4806,  2.1043, -1.0114, -0.0605, -0.4340,
          -1.1816, -1.3028],
         [-0.2313, -1.6736,  1.1868,  0.1514,  1.2037,  0.6815,  0.5119, -0.8219,
          -1.0702,  0.9774],
         [-0.7753,  0.4698, -2.0141, -0.4705, -0.6935, -0.5498,  0.6422,  1.5470,
           0.3227,  0.7972],
         [-2.7740,  0.9726, -0.0049, -1.2102,  2.8382, -0.5756,  1.6773,  0.1653,
           0.0534,  0.0058],
 

In [17]:
expert = nn.Linear(10, 10)

In [18]:
# 当前专家需要计算5个token
export_output = expert(expert_input)
export_output,export_output.shape

(tensor([[ 0.6423, -0.6202, -0.0127,  0.7240, -0.4062, -0.1476, -0.8556, -0.2711,
          -0.7112,  0.1809],
         [ 0.5604, -0.9486,  0.4157,  0.1808, -0.7739, -0.4275, -0.6011, -0.1783,
          -0.1547,  0.1917],
         [ 0.1614, -0.0955, -0.0219,  0.5253, -0.5164, -0.5552, -0.4339, -0.6135,
          -0.4151,  0.2187],
         [ 0.7932, -0.0317,  0.6402,  0.0976, -0.3972, -0.5325, -0.5507,  0.1384,
          -1.0871,  0.8516],
         [-0.5739,  0.4679,  0.0686,  0.2279, -0.1081, -0.6424, -0.7399, -0.6503,
          -0.1738,  0.2871],
         [ 0.4426, -0.0806,  0.0925, -0.1738,  0.2955, -0.0935,  0.0042,  0.2013,
           0.3963, -0.0284]], grad_fn=<AddmmBackward0>),
 torch.Size([6, 10]))

In [19]:
# 每个token对应专家分数或者权重
sparse_logits_flat,sparse_logits_flat.shape,expert_mask_flat

(tensor([[0.0000, 0.0000, 0.3075, 0.6925],
         [0.0000, 0.0000, 0.2936, 0.7064],
         [0.0000, 0.0000, 0.5304, 0.4696],
         [0.0000, 0.0000, 0.4371, 0.5629],
         [0.4381, 0.0000, 0.0000, 0.5619],
         [0.0000, 0.0000, 0.3980, 0.6020],
         [0.0000, 0.4751, 0.0000, 0.5249],
         [0.5158, 0.0000, 0.4842, 0.0000],
         [0.0000, 0.0000, 0.3842, 0.6158],
         [0.0000, 0.0000, 0.3377, 0.6623],
         [0.0000, 0.0000, 0.3687, 0.6313],
         [0.6220, 0.0000, 0.0000, 0.3780],
         [0.5097, 0.0000, 0.0000, 0.4903],
         [0.4983, 0.0000, 0.0000, 0.5017],
         [0.0000, 0.0000, 0.5665, 0.4335],
         [0.4773, 0.0000, 0.5227, 0.0000]], grad_fn=<ViewBackward0>),
 torch.Size([16, 4]),
 tensor([False, False, False, False,  True, False, False,  True, False, False,
         False,  True,  True,  True, False,  True]))

In [20]:
# 提取门控分数
gate_scores = sparse_logits_flat[expert_mask_flat, 0]
gate_scores,gate_scores.shape

(tensor([0.4381, 0.5158, 0.6220, 0.5097, 0.4983, 0.4773],
        grad_fn=<IndexBackward0>),
 torch.Size([6]))

In [21]:
gate_scores = gate_scores.unsqueeze(1)
gate_scores,gate_scores.shape

(tensor([[0.4381],
         [0.5158],
         [0.6220],
         [0.5097],
         [0.4983],
         [0.4773]], grad_fn=<UnsqueezeBackward0>),
 torch.Size([6, 1]))

In [67]:
weighted_output = export_output * gate_scores
export_output,weighted_output,weighted_output.shape

(tensor([[ 2.2763e-01, -6.6976e-01,  3.5573e-02,  7.3793e-02,  1.0876e+00,
          -5.6309e-01,  1.7597e-01, -4.7608e-01,  1.8169e-01, -3.3662e-01],
         [ 4.1032e-01, -5.3572e-01, -1.9546e+00, -3.2027e-01,  1.4034e+00,
          -1.2927e+00, -7.8958e-01, -8.3510e-01,  1.0911e+00, -9.6988e-01],
         [ 6.9326e-01,  3.3735e-01,  1.9023e-01,  1.0221e-01,  7.8824e-01,
           1.8935e-01,  1.8323e-01, -7.9960e-01,  6.3797e-01, -8.9355e-01],
         [ 2.7632e-01, -3.2140e-01, -6.2477e-02, -1.4784e-02,  1.0111e+00,
          -4.6016e-01,  1.4112e+00, -1.1115e-01,  1.0467e+00, -7.5915e-01],
         [-4.8984e-01, -2.6551e-01, -4.2119e-02, -9.8407e-01,  1.5780e-01,
           6.4319e-01, -1.0525e+00,  3.7018e-01,  9.5109e-01,  9.7007e-01],
         [-9.2653e-01, -3.2341e-01, -1.8357e-01, -9.9574e-01,  2.1067e-01,
          -5.2553e-01, -1.0058e+00,  4.8491e-01, -2.2569e-01, -1.9420e-01],
         [-8.2126e-01, -6.3031e-02, -8.7859e-02, -1.8947e-01,  9.9473e-01,
           1.1585e+

In [79]:
final_outputs[expert_mask].shape

torch.Size([11, 10])

In [77]:
final_outputs[expert_mask] += weighted_output
final_outputs,final_outputs.shape,expert_mask,expert_mask.shape

(tensor([[[ 2.5558e-01, -7.5201e-01,  3.9942e-02,  8.2855e-02,  1.2211e+00,
           -6.3224e-01,  1.9758e-01, -5.3455e-01,  2.0400e-01, -3.7796e-01],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
            0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 8.2849e-01, -1.0817e+00, -3.9466e+00, -6.4667e-01,  2.8336e+00,
           -2.6101e+00, -1.5943e+00, -1.6862e+00,  2.2031e+00, -1.9583e+00],
          [ 1.3867e+00,  6.7480e-01,  3.8051e-01,  2.0445e-01,  1.5767e+00,
            3.7876e-01,  3.6651e-01, -1.5994e+00,  1.2761e+00, -1.7874e+00],
          [ 5.5800e-01, -6.4903e-01, -1.2616e-01, -2.9855e-02,  2.0417e+00,
           -9.2924e-01,  2.8498e+00, -2.2445e-01,  2.1136e+00, -1.5330e+00],
          [-9.5677e-01, -5.1861e-01, -8.2268e-02, -1.9221e+00,  3.0821e-01,
            1.2563e+00, -2.0557e+00,  7.2304e-01,  1.8577e+00,  1.8948e+00],
          [-9.5009e-01, -3.3163e-01, -1.8823e-01, -1.0211e+00,  2.1603e-01,
      

In [22]:
gate_logit,gate_logit.shape


(tensor([[-0.7282, -0.8363,  0.1601,  0.9720],
         [-0.6245, -0.4649, -0.0513,  0.8264],
         [-0.6369, -0.2655,  0.4256,  0.3038],
         [-0.5963, -1.6130, -0.0584,  0.1944],
         [ 0.1214, -0.0660, -0.1532,  0.3703],
         [-0.6829, -0.7744, -0.3790,  0.0347],
         [-1.2188,  0.2750, -0.1770,  0.3746],
         [ 0.7845, -1.1465,  0.7213,  0.6494],
         [-0.4878, -1.1083,  0.7122,  1.1838],
         [-0.6631,  0.3346,  0.9861,  1.6595],
         [-0.9269, -0.6006,  0.1411,  0.6790],
         [ 0.4634, -0.6352, -0.0380, -0.0348],
         [ 0.8594,  0.3049, -0.0748,  0.8208],
         [-0.1530, -0.9408, -0.1673, -0.1462],
         [-0.5209, -0.6614,  0.5353,  0.2677],
         [ 0.3297,  0.1474,  0.4207, -0.1707]], grad_fn=<ViewBackward0>),
 torch.Size([16, 4]))

In [23]:
routing_weights = F.softmax(gate_logit, dim=-1)
routing_weights,routing_weights.shape

(tensor([[0.1020, 0.0915, 0.2480, 0.5585],
         [0.1217, 0.1428, 0.2160, 0.5195],
         [0.1265, 0.1834, 0.3660, 0.3241],
         [0.1894, 0.0685, 0.3244, 0.4177],
         [0.2583, 0.2142, 0.1963, 0.3313],
         [0.1881, 0.1716, 0.2549, 0.3855],
         [0.0757, 0.3372, 0.2146, 0.3725],
         [0.3381, 0.0490, 0.3174, 0.2954],
         [0.0983, 0.0528, 0.3262, 0.5227],
         [0.0523, 0.1419, 0.2722, 0.5337],
         [0.0973, 0.1348, 0.2831, 0.4848],
         [0.3927, 0.1309, 0.2378, 0.2386],
         [0.3414, 0.1961, 0.1341, 0.3284],
         [0.2901, 0.1319, 0.2859, 0.2920],
         [0.1440, 0.1251, 0.4140, 0.3168],
         [0.2829, 0.2357, 0.3098, 0.1715]], grad_fn=<SoftmaxBackward0>),
 torch.Size([16, 4]))

In [24]:
_, selected_experts = torch.topk(routing_weights, 2, dim=-1)
selected_experts,selected_experts.shape

(tensor([[3, 2],
         [3, 2],
         [2, 3],
         [3, 2],
         [3, 0],
         [3, 2],
         [3, 1],
         [0, 2],
         [3, 2],
         [3, 2],
         [3, 2],
         [0, 3],
         [0, 3],
         [3, 0],
         [2, 3],
         [2, 0]]),
 torch.Size([16, 2]))

In [25]:
expert_mask = torch.nn.functional.one_hot(selected_experts, 4)
expert_mask,expert_mask.shape

(tensor([[[0, 0, 0, 1],
          [0, 0, 1, 0]],
 
         [[0, 0, 0, 1],
          [0, 0, 1, 0]],
 
         [[0, 0, 1, 0],
          [0, 0, 0, 1]],
 
         [[0, 0, 0, 1],
          [0, 0, 1, 0]],
 
         [[0, 0, 0, 1],
          [1, 0, 0, 0]],
 
         [[0, 0, 0, 1],
          [0, 0, 1, 0]],
 
         [[0, 0, 0, 1],
          [0, 1, 0, 0]],
 
         [[1, 0, 0, 0],
          [0, 0, 1, 0]],
 
         [[0, 0, 0, 1],
          [0, 0, 1, 0]],
 
         [[0, 0, 0, 1],
          [0, 0, 1, 0]],
 
         [[0, 0, 0, 1],
          [0, 0, 1, 0]],
 
         [[1, 0, 0, 0],
          [0, 0, 0, 1]],
 
         [[1, 0, 0, 0],
          [0, 0, 0, 1]],
 
         [[0, 0, 0, 1],
          [1, 0, 0, 0]],
 
         [[0, 0, 1, 0],
          [0, 0, 0, 1]],
 
         [[0, 0, 1, 0],
          [1, 0, 0, 0]]]),
 torch.Size([16, 2, 4]))

In [26]:
tokens_per_expert = torch.mean(expert_mask.float(), dim=0)
tokens_per_expert,tokens_per_expert.shape

(tensor([[0.1875, 0.0000, 0.1875, 0.6250],
         [0.1875, 0.0625, 0.5000, 0.2500]]),
 torch.Size([2, 4]))

In [27]:
router_prob_per_expert = torch.mean(routing_weights, dim=0)
router_prob_per_expert,router_prob_per_expert.shape

(tensor([0.1937, 0.1505, 0.2750, 0.3808], grad_fn=<MeanBackward1>),
 torch.Size([4]))

In [35]:
topk_idx_for_aux_loss = torch.tensor([
    [1, 0, 2, 3, 1, 0],  # 样本0：每个 Token 的 2 个专家索引
    [0, 1, 3, 2, 0, 2]   # 样本1
])
topk_idx_for_aux_loss

tensor([[1, 0, 2, 3, 1, 0],
        [0, 1, 3, 2, 0, 2]])

In [36]:
ce = torch.tensor([
    [0.0, 0.0, 0.0, 0.0],  # 样本0
    [0.0, 0.0, 0.0, 0.0]   # 样本1
])
ce

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.]])

In [37]:
ce.scatter_add_(1,topk_idx_for_aux_loss,torch.ones(2,6))

tensor([[2., 2., 1., 1.],
        [2., 1., 2., 1.]])